In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import os
import json
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.high_level import extract_pages
from pdfminer.layout import LAParams
from pdfminer.layout import LTTextContainer
from json import JSONEncoder
from collections import namedtuple



## Extracting from JSON from DNB Eiendom Salgsoppgaver

we need to find:<br>

price in nok - convert to int strings that start with "kr" DONE <br>
location adress and kommune DONE <br>
buldingyear- find any 4 letter string that can be converted to int and that is below 2030 DONE <br> 
energy label - color and type DONE<br>
number of rooms DONE <br>
area DONE <br>
heating DONE



In [2]:
class Bolig:
        prisantydning = []
        byggeaar = []
        bruksareal = []
        number_of_rooms = []
        heating = []
        location = []
        energy_label = []
        property_type_description = []

In [3]:
class BoligEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

def customBoligDecoder(boligDict):
    return namedtuple('X', boligDict.keys())(*boligDict.values())

In [5]:
def checkstr(s):
        try:
            out = int(s)
            return True
        except:
            return False
        
def findPrice(boligelement):
    prices = [] 
    for el in boligelement:
            pris = [x.replace("kr","") for x in el if str.startswith(x, "kr ")]
            pris = [x.replace(" ","") for x in pris ]
            pris = [int(x) for x in pris if checkstr(x) == True]
            if pris != []:
                prices.extend(pris)    
    return prices

    
    
def findArea(boligelement):
    Areas = [] 
    for el in boligelement:
            if "bruksareal" in el:
                possbileArea = el[el.index("bruksareal")+1]
                if str.endswith(possbileArea,"m²"):
                    possbileArea = possbileArea.replace(" m²","")
                    if checkstr(possbileArea) == True:
                        Areas.append(int(possbileArea))
    return list(set(Areas))

def findBuildingYear(boligelement):
    buildingYears = [] 
    for el in boligelement:
        if "byggeår" in el:
            possbileYear = el[el.index("byggeår")+1]
            if len(possbileYear) == 4 and checkstr(possbileYear) == True:
                possbileYear = int(possbileYear)
                if possbileYear > 1800 and possbileYear <= 2030:
                    buildingYears.append(possbileYear)
    return buildingYears

def findBuildingAddresse(boligelement):
    location = {"adresse":[],"kommune":[]}
    for el in boligelement:
        if "adresse" in el:
            location["adresse"].append(el[el.index("adresse")+1])
        if "kommune" in el:
            location["kommune"].append(el[el.index("kommune")+1])
    ## fix formatting
    if location["adresse"] != []:
        location["adresse"] = [str.strip(x) for x in location["adresse"] if x != '']
        location["adresse"] = list(set(location["adresse"]))
        if len(location["adresse"]) > 1:
            location["adresse"] = location["adresse"][-1]
        elif len(location["adresse"]) == 1:
            location["adresse"] = location["adresse"][0]
    if location["kommune"] != []:
        location["kommune"] = location["kommune"][-1]
    answer = []
    answer.append(location["adresse"])
    answer.append(location["kommune"])
    if answer[0] == [] or answer[1] == []:
        return None
    return ", ".join(answer)


def findEnergyLabel(boligelement):
    energyLabel = [] 
    for el in boligelement:
            if "energikarakter" in el:
                karakter = el[el.index("energikarakter")+1]
                if  str.strip(karakter) != "ikke angitt" and str.strip(karakter) != '':
                    energyLabel.append(karakter)
            if "energifarge" in el:
                farge = el[el.index("energifarge")+1]
                if str.strip(farge) != "ikke angitt" and str.strip(farge) != '':
                    energyLabel.append(farge)
    if energyLabel == []:
        return None
    return energyLabel


def findNumberOfRooms(boligelement):
    numberOfRooms = [] 
    for el in boligelement:
            if "antall rom" in el:
                nRooms = el[el.index("antall rom")+1]
                if nRooms != '' and checkstr(nRooms) == True:
                    numberOfRooms.append(int(nRooms))
    if numberOfRooms == []:
        return None
    return numberOfRooms
               
def Mapper(fileJson):
    bolig = Bolig()
    bolig.prisantydning = fileJson["prisantydning"]
    bolig.byggeaar = fileJson["byggeaar"]
    bolig.bruksareal = fileJson["bruksareal"]
    bolig.number_of_rooms = fileJson["number_of_rooms"]
    bolig.heating = fileJson["heating"]
    bolig.location = fileJson["location"]
    bolig.energy_label = fileJson["energy_label"]
    bolig.property_type_description = fileJson["property_type_description"]
    return bolig

Heatingtype = {
"district heating": ["fjernvarmeanlegg","fjernvarme","fjernvarmenettet","fjernvarmeanlegget", "fjernvarmetilknytning", "fjernvarme", "fjernvarmesentral","fjernvarmen"],

 "radiators": [ "radiatorer", "panelovner", "radiator", "panel","panelovn", "panelover", "radiatorovner","radiatorovn", "radiatorene", "stråleovner","varmeovn" ],
 
"floor heating": ["gulvvarme", "varmekabler", "varmekabel", "gulvvarmen", "gullvarme", "gulvarme","varmekabelr","varmkabler"],

"electrical": ["elektriske", "elektrisk", "elekrtrisk", "elektrisitert", "elfyrkjele", "el‑kjele","elkjele","elvarme","elgulvvarme" ],

"heat pump": ["varmepumpe","varmepumpen","varmepumper","varmepumpeanlegg","avkastvarmepumpe","varmepumpesystem","luftvarmepumpe","bergvarmepumpen","varmepumpumpe","varmepumpene"],

"central heating": ["sentralfyring", "sentralvarmeanlegg", "sentralvarme", "sentralanlegg","fellesanlegg", "varmesentral", "varmeanlegget", "sentralfyr", "fyringsanlegg","fyringsanlegget"],

"hot water heating": ["vannbårent","vannboren","vannbårenvarme","luft‑til‑vann","lufttilvann"],

"hot air heating" : ["varmluft","luft‑til‑luft", "lufttilluft", "luft-luft","luftbåren","luft‑luft" ],

"energy saving measures": ["varmefolie", "vindfang", "varmegjenvinning","varmegjenvinner", "energigjenvinning","varmematte","varmekonvektor", "varmeveksler", "jordvarme" ,"isolerglassvinduer","energibrønner", "varmefolien","gjenvinningsvarmepumpe","solcellepanel","varmegjenvinneren","depronfolie", "solcellepaneler"],

"bioenergy": ["vedfyring", "vedovn", "peis", "peisovn","bioenergi","kamin", "klebersteinspeis", "peisen", "kakkelovn","fyrkjele","vedfyrt","biofyringsolje", "bioparafin","biodrivstoff","vedovner","vedovns","peisovner","designpeis","pelletsovn","elementpeis","klebersteinovn","vedoppfyring", "gruepeis", "vedfyringsovn", "vedkamin", "klebersteinsovn","vedovnen"],


"oil and gas": ["gasspeis", "parafinovn", "oljefyring", "gasspeiser","gasstank","gassanlegg", "oljekamin","fossil", "olje", "parafin","parafinfyringsanlegg","oljetanker","oljetank","parafintank","mineralolje","oljefyrte", "oljetanken","gassovn","oljefyr","oljefyringsanlegg","oljefyrt", "parafinbrenner","gassanlegget" ]}

def FindHeating(input):
    heating = []
    if input  != []:
        el = input[0]
        if el == []:
            el = input [1]
        el = " ".join(el)
        el = el.split(" ")
        el = [str.strip(x).replace(".","") for x in el]
        el = [x.replace(",","") for x in el]
        el = [x.replace("/","") for x in el]
        el = [x.replace(")","") for x in el]
        el = [x.replace("(","") for x in el]
        el = [x.replace(r"\\","") for x in el]
        el = [x.replace(r"-","") for x in el]
        el = [x.replace(r"|","") for x in el]
        el = [x.replace(":","") for x in el]
        el = [x.replace('"',"") for x in el]
        heating.extend(el)
        
    return heating

def get_key(val, my_dict):
    for key, value in my_dict.items():
         if val in value:
             return key
    return None


    
    
            

In [22]:
i = 1
folderUrl = "./DNB/JSON/Untreated/Tomansbolig/"
folderOutputUrl = "./DNB/JSON/Treated/"
counterPrices = 0
counterWithoutPrices = 0
for file in os.listdir(folderUrl):
    print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        newBolig = Bolig()
        newBolig.prisantydning = []
        newBolig.byggeaar = []
        newBolig.bruksareal = []
        newBolig.number_of_rooms = []
        newBolig.heating = []
        newBolig.location = []
        newBolig.energy_label = []
        newBolig.property_type_description = ["tomansbolig"]
        ## how many empty string there are in each property
        ##find price
        prices = []
        ##find area
        Areas = []
        ### find building year
        BuildingYear = []
        ## Adresse
        Addresse = []
        ## Energy Label
        EnergyLabel = []
        ## Number Of Rooms
        NumberOfRooms = []
        
        ### Section Prisantydning
        thisprice = findPrice(boligelement=bolig.prisantydning)
        if thisprice != None:
            prices.extend(thisprice)
        thisarea = findArea(boligelement=bolig.prisantydning)
        if thisarea != []:
            Areas.extend(thisarea)
        thisbuildingYear = findBuildingYear(boligelement=bolig.prisantydning)
        if thisbuildingYear != []:
            BuildingYear.extend(thisarea)
        thisAddresse = findBuildingAddresse(boligelement=bolig.prisantydning)
        if thisAddresse != None and thisAddresse not in Addresse:
            Addresse.append(thisAddresse)
        thisEnergyLabel = findEnergyLabel(boligelement=bolig.prisantydning)
        if thisEnergyLabel != None and thisEnergyLabel not in EnergyLabel:
            EnergyLabel.append(thisEnergyLabel)
        thisNumberOfRooms = findNumberOfRooms(boligelement=bolig.prisantydning)
        if thisNumberOfRooms != None and thisNumberOfRooms not in NumberOfRooms:
            NumberOfRooms.extend(thisNumberOfRooms)
        
             
            
            
        ### Byggeår section
        thisprice = findPrice(boligelement=bolig.byggeaar)
        if thisprice != None:
            prices.extend(thisprice)
        thisarea = findArea(boligelement=bolig.byggeaar)
        if thisarea != []:
            Areas.extend(thisarea)
        thisbuildingYear = findBuildingYear(boligelement=bolig.byggeaar)
        if thisbuildingYear != []:
            BuildingYear.extend(thisbuildingYear)
        thisAddresse = findBuildingAddresse(boligelement=bolig.byggeaar)
        if thisAddresse != None and thisAddresse not in Addresse:
            Addresse.append(thisAddresse)
        thisEnergyLabel = findEnergyLabel(boligelement=bolig.byggeaar)
        if thisEnergyLabel != None and thisEnergyLabel not in EnergyLabel:
            EnergyLabel.append(thisEnergyLabel)
        thisNumberOfRooms = findNumberOfRooms(boligelement=bolig.byggeaar)
        if thisNumberOfRooms != None and thisNumberOfRooms not in NumberOfRooms:
            NumberOfRooms.extend(thisNumberOfRooms)
        
        
            
            
        ### Bruksareal Section
        thisprice = findPrice(boligelement=bolig.bruksareal)
        if thisprice != None:
            prices.extend(thisprice)
        thisarea = findArea(boligelement=bolig.bruksareal)
        if thisarea != []:
            Areas.extend(thisarea)
        thisbuildingYear = findBuildingYear(boligelement=bolig.bruksareal)
        if thisbuildingYear != []:
           BuildingYear.extend(thisbuildingYear)
        thisAddresse = findBuildingAddresse(boligelement=bolig.bruksareal)
        if thisAddresse != None and thisAddresse not in Addresse:
            Addresse.append(thisAddresse)
        thisEnergyLabel = findEnergyLabel(boligelement=bolig.bruksareal)
        if thisEnergyLabel != None and thisEnergyLabel not in EnergyLabel:
            EnergyLabel.append(thisEnergyLabel)
        thisNumberOfRooms = findNumberOfRooms(boligelement=bolig.bruksareal)
        if thisNumberOfRooms != None and thisNumberOfRooms not in NumberOfRooms:
            NumberOfRooms.extend(thisNumberOfRooms)
            
        ## Number of Rooms Section
        thisprice = findPrice(boligelement=bolig.number_of_rooms)
        if thisprice != None:
            prices.extend(thisprice)
        thisarea = findArea(boligelement=bolig.number_of_rooms)
        if thisarea != []:
            Areas.extend(thisarea)
        thisbuildingYear = findBuildingYear(boligelement=bolig.number_of_rooms)
        if thisbuildingYear != []:
            BuildingYear.extend(thisbuildingYear)    
        thisAddresse = findBuildingAddresse(boligelement=bolig.number_of_rooms)
        if thisAddresse != None and thisAddresse not in Addresse:
            Addresse.append(thisAddresse)
        thisEnergyLabel = findEnergyLabel(boligelement=bolig.number_of_rooms)
        if thisEnergyLabel != None and thisEnergyLabel not in EnergyLabel:
            EnergyLabel.append(thisEnergyLabel)
        thisNumberOfRooms = findNumberOfRooms(boligelement=bolig.number_of_rooms)
        if thisNumberOfRooms != None and thisNumberOfRooms not in NumberOfRooms:
            NumberOfRooms.extend(thisNumberOfRooms)
            
        ## Heating Section
        thisprice = findPrice(boligelement=bolig.heating)
        if thisprice != None:
            prices.extend(thisprice)
        thisarea = findArea(boligelement=bolig.heating)
        if thisarea != []:
            Areas.extend(thisarea)
        thisbuildingYear = findBuildingYear(boligelement=bolig.heating)
        if thisbuildingYear != []:
            BuildingYear.extend(thisbuildingYear)
        thisAddresse = findBuildingAddresse(boligelement=bolig.heating)
        if thisAddresse != None and thisAddresse not in Addresse:
            Addresse.append(thisAddresse)
        thisEnergyLabel = findEnergyLabel(boligelement=bolig.heating)
        if thisEnergyLabel != None and thisEnergyLabel not in EnergyLabel:
            EnergyLabel.append(thisEnergyLabel)
        thisNumberOfRooms = findNumberOfRooms(boligelement=bolig.heating)
        if thisNumberOfRooms != None and thisNumberOfRooms not in NumberOfRooms:
            NumberOfRooms.extend(thisNumberOfRooms)
        
        Heating = FindHeating(bolig.heating)
        heating_type = []
        for word in Heating:
            type = get_key(word,Heatingtype)
            if type != None and type not in heating_type:
                heating_type.append(type)
                continue
        newBolig.heating = heating_type
            
        ##Location
        thisprice = findPrice(boligelement=bolig.location)
        if thisprice != None:
            prices.extend(thisprice)
        thisarea = findArea(boligelement=bolig.location)
        if thisarea != []:
            Areas.extend(thisarea)
        thisbuildingYear = findBuildingYear(boligelement=bolig.location)
        if thisbuildingYear != []:
            BuildingYear.extend(thisbuildingYear)
        thisAddresse = findBuildingAddresse(boligelement=bolig.location)
        if thisAddresse != None and thisAddresse not in Addresse:
            Addresse.append(thisAddresse)
        thisEnergyLabel = findEnergyLabel(boligelement=bolig.location)
        if thisEnergyLabel != None and thisEnergyLabel not in EnergyLabel:
            EnergyLabel.append(thisEnergyLabel)
        thisNumberOfRooms = findNumberOfRooms(boligelement=bolig.location)
        if thisNumberOfRooms != None and thisNumberOfRooms not in NumberOfRooms:
            NumberOfRooms.extend(thisNumberOfRooms)
            
        ## EnergyLabel Section
        thisprice = findPrice(boligelement=bolig.energy_label)
        if thisprice != None:
            prices.extend(thisprice)
        thisarea = findArea(boligelement=bolig.energy_label)
        if thisarea != []:
            Areas.extend(thisarea)
        thisbuildingYear = findBuildingYear(boligelement=bolig.energy_label)
        if thisbuildingYear != []:
            BuildingYear.extend(thisbuildingYear)
        thisAddresse = findBuildingAddresse(boligelement=bolig.energy_label)
        if thisAddresse != None and thisAddresse not in Addresse:
            Addresse.append(thisAddresse)
        thisEnergyLabel = findEnergyLabel(boligelement=bolig.energy_label)
        if thisEnergyLabel != None and thisEnergyLabel not in EnergyLabel:
            EnergyLabel.append(thisEnergyLabel)
        thisNumberOfRooms = findNumberOfRooms(boligelement=bolig.energy_label)
        if thisNumberOfRooms != None and thisNumberOfRooms not in NumberOfRooms:
            NumberOfRooms.extend(thisNumberOfRooms)
            
        ## Numer of Rooms
        NumberOfRooms = list(set(NumberOfRooms))
        if NumberOfRooms != []:
            newBolig.number_of_rooms.extend(NumberOfRooms)
            
        ##Adresse
        if Addresse != []:
            newBolig.location.extend(Addresse)
                    
        
        ## Byggeår
        BuildingYear= list(set(BuildingYear))
        if BuildingYear != []:
            newBolig.byggeaar.extend(BuildingYear)

        ## Areal
        Areas = list(set(Areas))
        if Areas != []:
            newBolig.bruksareal.extend(Areas)
        
        ##Pris          
        if prices != []:
            newBolig.prisantydning = [max(prices)]
        
        ## Energy Label
        if EnergyLabel != []:
            newBolig.energy_label.extend(EnergyLabel)
            
        ##Print ut
        #print(f"areal {newBolig.bruksareal}")
        #print(f"byggeår {newBolig.byggeaar}")
        #print(f"pris {newBolig.prisantydning}")
        #print(f"adresse {newBolig.location}")
        #print(f"Energy Label {newBolig.energy_label}")
        #print(f"NumberOfRooms {newBolig.number_of_rooms}")
        #print(f"Heating type {newBolig.heating}")
        
        jsonStr = json.dumps(newBolig.__dict__,indent=4,sort_keys=True,ensure_ascii=False)
        with open(folderOutputUrl + "HeatingDataJuly2022.json", 'a', encoding="utf-8") as outfile:
                outfile.write(jsonStr)
                outfile.write(",")
                outfile.close()

        i+=1


document number 1 0a4baee6-232c-4229-0e7b-08da0e35b67c.json
document number 2 0d7ff2d5-8311-49ec-dd27-08da566e3a83.json
document number 3 0f080f82-9ac3-4386-efdd-08d8c384d651.json
document number 4 102884a2-0483-4d37-31ad-08da57710584.json
document number 5 10daf0c7-4ef6-4d61-b9e6-08da4b22dada.json
document number 6 1a3ec1b3-ae76-41c2-57fd-08d9cc835aa2.json
document number 7 1bffa2ec-f0bb-4d77-a90d-08da46410a37.json
document number 8 1c12b95b-23c6-4c9e-d3e9-08d99581a3c7.json
document number 9 1c1e4e77-8a0a-4a4a-fd82-08da4e8ae43a.json
document number 10 1c20f570-5c83-4ae2-e6e4-08da08f75b13.json
document number 11 1ff3f7fd-2ed4-4fa9-93c9-08d67a995e7d.json
document number 12 227d6e3c-0c7a-45d3-5a75-08da43f35447.json
document number 13 2510ff9c-1331-4333-4c32-08da5e48966e.json
document number 14 2c1a813f-e207-4527-5861-08d92a565ea8.json
document number 15 2c33e833-64db-4088-a545-08da38a94ca7.json
document number 16 2c8263f3-beac-48ba-c6b2-08da5d905ee9.json
document number 17 2db3a6b9-1c06-

In [114]:
## heating
i = 1
folderUrl = "./DNB/JSON/Untreated/Rekkehus/"
withValues = 0
wihtoutValues = 0

for file in os.listdir(folderUrl):
    print(f"document number {i} {file}")
    with open(folderUrl + file, 'r', encoding = "utf-8") as f:
        Jsonfile = json.load(f)
        bolig = Mapper(Jsonfile)
        
        
        print(heating_type)
        i += 1


document number 1 0148767a-a9aa-43b7-71dd-08da511399c3.json
['electrical', 'floor heating']
document number 2 0ac6c21b-3270-47d2-545c-08da107bf601.json
['electrical', 'energy saving measures']
document number 3 0ce8c6fb-2694-4c35-a28c-08da4321c1c3.json
['electrical', 'radiators', 'floor heating', 'heat pump', 'bioenergy']
document number 4 0d0b401a-de44-41ec-3000-08da3fd57391.json
['electrical', 'radiators', 'floor heating', 'heat pump']
document number 5 0fe50b23-5c71-4657-9817-08da3971744c.json
['bioenergy', 'electrical', 'floor heating', 'energy saving measures']
document number 6 107a2a1b-9319-43da-05d3-08da4e3fe05d.json
['bioenergy']
document number 7 10c7fd1b-f5f0-44ae-9903-08da1c8069b2.json
['bioenergy']
document number 8 1a264cf8-32d2-46de-0427-08da55bbf5f2.json
['bioenergy', 'electrical']
document number 9 2408923b-20b0-423d-b0d2-08da4fdacd99.json
['radiators', 'bioenergy']
document number 10 243998cb-4bc3-46f5-a4b5-08da50a2927c.json
['heat pump']
document number 11 24615ada-f